<a href="https://colab.research.google.com/github/chengchun100/start2/blob/main/2_danmu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request as req
url = "https://github.com/Elwing-Chou/nanshan1115/raw/main/danmu.txt"
req.urlretrieve(url, "damu.txt")

In [ ]:
#要把檔案中的字串改為list和字典型態，所以配合json
import json
f = open("damu.txt", "r", encoding="utf-8")
danmus = json.load(f)
f.close()
danmus

In [ ]:
#pd.DataFrame型態:表格
import pandas as pd
df = pd.json_normalize(danmus)
df

In [ ]:
# matplotlib: 基礎的畫圖函式庫
# seaborn: 進階畫圖函式庫
import matplotlib.pyplot as plt
import seaborn as sns
#bins是間隔的意思，kde是核修勻，可以看連續時間分布下實際上的熱度
sns.histplot(x=df["time"], bins=100, kde=True)

In [ ]:
# 1. 表格篩選操作,DataFrame[跟你資料筆數依樣多的True/False]
test = pd.DataFrame([
    [1,2],
    [3,4],
    [5,6]
])
test[[True,False,True]]

In [ ]:
#pandas第一種型態: DataFrame
#第二種型態: Series(只取出一列或是一行)
#如果取filter會用到Python內建功能，所以不能拿它當變數
#連接兩個篩選條件的方法:而且(&) 或者(|)
fil = (df["time"] > 65000) & (df["color"] == "#FFFFFF")
df[fil]

In [ ]:
#2.轉換操作: apply
#型態:字典, 操作[key]
#函式也是種型態:函式 e.g.int，小括號是操作出這個功能:(4.2)，所以兩者是可以分開的
b = int
b(4.2)

In [ ]:
def square(n):
    ans = n * n
    return ans
square(4)

In [ ]:
#接下來示範對表格中的每一個元素都做處理的操作
#首先示範Series的轉換操作，建立一個Series後，應用前面寫的平方功能
#apply會對表格中每一個元素都進行所定義的功能操作一遍
test = pd.Series([1,2,3])
test.apply(square)

In [38]:
#這部分的code是先前在介紹jieba時所用的coding
import urllib.request as req
import jieba

url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
req.urlretrieve(url, "dict.big")
jieba.set_dictionary("dict.big")

In [ ]:
#將表格中每一則彈幕的內容進行「分詞」，運用到剛剛介紹的apply(功能)
def func(s):
    return " ".join(jieba.cut(s))
df["text"] = df["text"].apply(func)
df

In [ ]:
#把彈幕中所有的分詞作輿情分析，並計算出各詞彙的TF-IDF得分
import jieba.analyse
article = "\n".join(df["text"])
result = jieba.analyse.extract_tags(article, topK=50, withWeight=True)
#儲存完result後，透過pandas的功能，將result轉成dataframe
keywords = pd.DataFrame(result, columns=["keyword","tfidf"])
keywords

In [50]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 在執行繪圖時，常常會遇到字體變成豆腐字的狀況，代表程式顯示不出來
# 這一段程式碼有需要直接複製貼上就好了，不用背起來
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

url = "https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download"
req.urlretrieve(url, "TaipeiSansTCBeta-Regular.ttf")
fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

title:標題
axis:軸
label:軸標籤
ticks:軸刻度
legend:圖例

In [ ]:
#plt.figure(figsize=[寬度,高度])
plt.figure(figsize=[15,10])
sns.barplot(x=keywords["keyword"],
      y=keywords["tfidf"])
#由於x軸的keywords太擠了，所以要旋轉字體角度，讓每個關鍵字都能被看清楚
plt.xticks(rotation=45)

In [ ]:
# DataFrame轉換操作
# axis=0 or 1
def func(x):
    score = x["國文"] * 0.7 + x["數學"] * 0.3
    return score

test = pd.DataFrame([
    [90,60],
    [80,70]
], columns=["國文","數學"])
test["平均"] = test.apply(func, axis=1)
test

In [ ]:
# 找出所有彈幕有大哥，且彈幕顏色為白色(#FFFFFF)的訊息
def func(x):
    if "大哥" in x["text"] and x["color"] == "#FFFFFF":
        return True
    else:
        return False
# 轉換結合篩選，先判斷每一筆的True, False，再結合到原資料上
fil = df.apply(func, axis=1)
df[fil]

In [ ]:
df["userid"].value_counts()

In [69]:
#找出每個userid發的頭兩個彈幕內容
#head是取頭幾筆資料，跟R語言一樣
head = df.groupby("userid").head(2)
#把處理完的檔案儲存成csv
head.to_csv("head.csv", encoding="utf-8")

In [70]:
#以titanic資料作為練習，準備用Pandas合併這兩份檔案
url = "https://github.com/Elwing-Chou/nanshan1115/raw/main/titanic/train.csv"
req.urlretrieve(url,"train.csv")
url = "https://github.com/Elwing-Chou/nanshan1115/raw/main/titanic/test.csv"
req.urlretrieve(url,"test.csv")

('test.csv', <http.client.HTTPMessage at 0x7e5770775900>)

In [77]:
#使用Pandas合併檔案，並將合併後檔案匯出
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")
combine = pd.concat([train,test], axis=0)
combine.to_csv("combine.csv",encoding="utf-8")